In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
#***********************************************************************
# WKP viewer: Projecties beta 0.2
# 
# Copyright (c) 2019, Cor Berrevoets, registax@gmail.com
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice, development funding notice, and this permission
# notice shall be included in all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
# 
#*********************************************************************/

import param, panel as pn, holoviews as hv, numpy as np, pandas as pd, hvplot.pandas, geopandas as gpd
from bokeh.models import ColumnDataSource, HoverTool, GeoJSONDataSource
from shapely.geometry import Point, LineString
from bokeh.plotting import figure, show
from bokeh.tile_providers import CARTODBPOSITRON
#hv.extension('bokeh')
#renderer = hv.renderer('bokeh')
pn.extension()

############################# LOAD SHAPEFILE
shape_wlpt = gpd.read_file('shapefiles/allcentroids.shp')
shape_wlpt.crs={'init': 'epsg:28992'}
# create Amfo X,Y from geometry to allow merging 
shape_wlpt['projectie_x']=shape_wlpt.geometry.x
shape_wlpt['projectie_y']=shape_wlpt.geometry.y

############################# LOAD OORDEEL DATA
#Read datasets
oordeel = pd.read_csv('oordeel2018.csv')

############################# LOAD MONITOR DATA
localdir='datasets/'

datafiles= [#'1.maatregelen_20140507.csv', 
            '4.monitoringprogramma_owl_20181016.csv.gz']

for index in range(len(datafiles)):
    locd=pd.read_csv(localdir+datafiles[index], sep=';', decimal='.', dtype=object, encoding='latin1')
    locd.columns = locd.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '_').str.replace(')', '')


#convert X,Y to floating points (normally obj after loading)
locd.geometriepunt_x_rd=locd.geometriepunt_x_rd.astype(float)
locd.geometriepunt_y_rd=locd.geometriepunt_y_rd.astype(float)
############################# SELECT DATA
#selection

#add all centroid points for projections to database (outer-merge)
merged=pd.merge(locd,shape_wlpt, left_on='representatief_waterlichaam_code', right_on='owmident', how='outer')

#remove any row where X or Y points are NaN
merged = merged[np.isfinite(merged['geometriepunt_x_rd'])]
merged = merged[np.isfinite(merged['geometriepunt_y_rd'])]
merged = merged[np.isfinite(merged['projectie_x'])]
merged = merged[np.isfinite(merged['projectie_y'])]

merged=pd.merge(oordeel,merged,left_on=['waterlichaam_identificatie','typering_omschrijving'],
                right_on=['representatief_waterlichaam_code','parameter_omschrijving'], how='right')

#remap the oordelen to color
omap= { 'oordeel':{ 'slecht':'red','ontoereikend':'orange','matig':'yellow',
                   'goed':'lightgreen','zeer goed':'blue','voldoet niet':'red','voldoet':'blue', np.nan:'white'} }
merged=merged.replace(omap)

#project centroids into webmercator(3857) from amfo (28992)
shape_wlpt.crs={'init': 'epsg:28992'}
shape_wlpt=shape_wlpt.to_crs(epsg=3857)
bounds=shape_wlpt.total_bounds

#collect all parameters
paramselect=merged['typering_omschrijving'].dropna().unique()
params=sorted(paramselect.tolist())
paramselect=merged['waterbeheerder_omschrijving'].dropna().unique()
paramsbeheer=['all']+sorted(paramselect.tolist())

#set lists for selectors
startup=True
class dataPlot(param.Parameterized):
    #global startup, merged, shape_wlpt, sourcewl, sourcewlM, lijn_source, sourcept, sourceptM , sourcelijn
    #definition of widget-based-parameters
    parameter = param.ObjectSelector(default='ammonium', objects=params)
    beheerder = param.ObjectSelector(default='all', objects=paramsbeheer)
    
    #dataselection 
    def get_data(self):
        global startup, merged, shape_wlpt, sourcewl, sourcewlM, lijn_source, sourcept,sourceptM, sourcelijn, sourcemeet
        #select data
        subset=merged.query('parameter_omschrijving==@self.parameter')   
        if self.beheerder!='all':
            subset=subset.query('waterbeheerder_omschrijving==@self.beheerder')   
        ############################# create set of projected circles
        #a=subset.copy()
        a=subset.query('ligt_in_waterlichaam_code!=representatief_waterlichaam_code')
        testlist=a[['representatief_waterlichaam_code','oordeel', 'waterbeheerder_omschrijving']]
        #del a
        shape_wl=gpd.GeoDataFrame(pd.merge(testlist,shape_wlpt,
                                           left_on="representatief_waterlichaam_code",right_on='owmident'))
        
        sourcewl = ColumnDataSource(data=dict(
                                x=list(shape_wl['geometry'].x), #map geometry
                                y=list(shape_wl['geometry'].y),
                                representatief_waterlichaam_code=list(shape_wl['owmident'] ),
                                representatief_waterlichaam_omschrijving=list(shape_wl['owmnaam']),
                                #colorl=list(shape_wlpt['marker_color']),
                                owmtyped=list(shape_wl['owmtype']),
                                beheerder=list(shape_wl['waterbeheerder_omschrijving']),
                                oordeel=list(shape_wl['oordeel'])
                                ))
        ########################### create set of not projected circles
        a=subset.copy()
        a=a.query('ligt_in_waterlichaam_code==representatief_waterlichaam_code')
        testlist=a[['ligt_in_waterlichaam_code','oordeel','waterbeheerder_omschrijving']]   
        #del a
        #testlist=testlist.rename( columns={0: "test"})
        shape_wlM=gpd.GeoDataFrame(pd.merge(testlist,shape_wlpt,
                                            left_on="ligt_in_waterlichaam_code",right_on='owmident'))
        sourcewlM = ColumnDataSource(data=dict(
                                x=list(shape_wlM['geometry'].x), #map geometry
                                y=list(shape_wlM['geometry'].y),
                                representatief_waterlichaam_code=list(shape_wlM['owmident'] ),
                                representatief_waterlichaam_omschrijving=list(shape_wlM['owmnaam']),
                                #colorl=list(shape_wlpt['marker_color']),
                                owmtyped=list(shape_wlM['owmtype']),
                                beheerder=list(shape_wlM['waterbeheerder_omschrijving']),
                                oordeel=list(shape_wlM['oordeel'])
                                ))

        ###########################create a set of lines between meetpunt - projected points and project into webmercator
         
        subset=gpd.GeoDataFrame(subset)
        meetpt=subset.copy()
        meetpt['geometry']=meetpt.apply(lambda x: Point([(x['geometriepunt_x_rd'], x['geometriepunt_y_rd'] )] ),axis=1)
        
        subset['geometry'] =  subset.apply(lambda x: LineString([(x['geometriepunt_x_rd'], x['geometriepunt_y_rd'] ) ,
                                                                 (x['projectie_x'], x['projectie_y'] )] ),axis=1)
        subset.crs={'init': 'epsg:28992'}
        subset=subset.to_crs(epsg=3857)
        meetpt.crs={'init': 'epsg:28992'}
        meetpt=meetpt.to_crs(epsg=3857)
        
        sourcemeet = ColumnDataSource(data=dict(
                                x=list(meetpt['geometry'].x), #map geometry
                                y=list(meetpt['geometry'].y),
                                
                                ))
        
        lijn_source = GeoJSONDataSource(geojson=subset.to_json())

    def save(self):
        pass
    def start_plot(self):
        global startup,  sourcewl, sourcewlM, lijn_source, sourcept,  sourceptM, sourcelijn, sourcemeetpt
        hover = HoverTool(tooltips=[
        ("waterlichaam", "@representatief_waterlichaam_code"),
        ("info", "@representatief_waterlichaam_omschrijving"),
        ("typering","@owmtyped"),
        ("meetpunt","@ligt_in_waterlichaam_code"),
        ("info","@meetpunt_omschrijving"),
         ("parameter","@parameter_omschrijving"),
            ("beheerder","@beheerder")
         ])
        p = figure(match_aspect=True, aspect_scale=1, title='Waterlichaam', 
                   tools=[hover, "box_zoom, reset, pan,lasso_select,box_select, wheel_zoom,tap"],
                    x_range=(bounds[0], bounds[2]), y_range=(bounds[1], bounds[3]),
                     x_axis_type="mercator", y_axis_type="mercator", width=700)
        p.xaxis.visible = False
        p.yaxis.visible = False
        p.add_tile(CARTODBPOSITRON, alpha=0.6)
        sourcept=sourcewl
        sourceptM=sourcewlM
        sourcelijn=lijn_source
        sourcemeetpt=sourcemeet
        startup=False
        p.cross(x='x', y='y', source=sourcemeetpt, size=6, fill_color='lightgreen',color='green', line_width=1, legend= "meetpunt", )
        
        p.circle(x='x', y='y', size=6, source=sourcept, legend='projectie', fill_color='white',
                 color='oordeel', line_width=2,alpha=0.8)

        p.circle(x='x', y='y', source=sourceptM, size=7, color='lightgray', fill_color='oordeel',
                        line_width=1, fill_alpha=0.9, legend='waterlichaam')

        
        
        project=p.multi_line('xs', 'ys', source=sourcelijn,  line_width=1, legend= "projectie-lijn")
        project.visible=False
        p.legend.location = "top_left"
        p.legend.click_policy="hide"
        return p

        
    @param.depends('parameter','beheerder', watch=True) #watch reacts to any change of paramA or param1
    def create_plot(self):
        global startup,  sourcewl, sourcewlM, lijn_source, sourcept,  sourceptM, sourcelijn, sourcemeetpt, p
        
        self.get_data()
        if startup==False:
            pass
            sourcept.data=sourcewl.data
            sourceptM.data=sourcewlM.data
            sourcelijn.geojson=lijn_source.geojson
            sourcemeetpt.data=sourcemeet.data
        if startup:
            p=self.start_plot()
            startup=False
            #sourcebestrijding=sourcebes
  
        p.title.text ="stof: {0} beheerders: {1}".format(self.parameter, self.beheerder)   
        return p

dataplot = dataPlot(name='')
plt=dataplot.create_plot()
desc = pn.pane.HTML("""
    <br><h3>Oordelen en projectie</h3><br>
    Overzicht van monitoring van chemie, gebaseerd op WKP gegevens uit 2018.
    Het oordeel is weergegeven in gevulde rood/blauwe cirkels indien gemeten,
    open cirkels geven aan waar gegevens op projectie zijn gebaseerd.
    """, width=250)

#paramcontrols can be called like this to allow carefull building of the gui
left = pn.panel(dataplot, parameters=['parameter'], width=200)
right = pn.panel(dataplot, parameters=['beheerder'], width=200)
pn.Row(pn.Column(desc,left, right), pn.Spacer(width=50),dataplot.create_plot)